# Data Collection
Please note that the following data collection and aggregation was done on 2/6/2022. I will be collecting this public data from SEC filings through finsight.com. The collection process took approximately 1 day to complete due to network speed.

In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import requests
import os
import sys

In [30]:
# create directory to hold data in location of this file
os.chdir(sys.path[0])
if not os.path.isdir('data'):
    os.mkdir('data')

os.chdir('data')

Must install chrome and the appropriate web driver version and change path to locate it below

In [31]:
driver = webdriver.Chrome('C:\Program Files\Google\Chrome-Driver\chromedriver.exe')
url = 'https://finsight.com/us-abs-loan-level-data-on-edgar'
driver.get(url)
driver.implicitly_wait(220)

<ipython-input-31-976fb71c061a>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\Program Files\Google\Chrome-Driver\chromedriver.exe')


In [32]:
# Filter datasets to only include auto tags
filter_input = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div[1]/div[1]/div/label/input")
filter_input.send_keys("AUTO")
filter_input.send_keys(Keys.ENTER)
driver.implicitly_wait(50)

<ipython-input-32-a9fdfc06d578>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  filter_input = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div[1]/div[1]/div/label/input")


In [33]:
# Iterate through each page, retrieving the download link <a> tag
# runs until manually exited. (or errors out due to no remaining pages)
while True:
    rows = driver.find_elements_by_class_name("DJAQA0Bgt6HD8NwtPxQv")
    for row in rows:
        try:
            link = row.find_element_by_xpath(".//td[5]/span[1]/span/span/a").get_attribute('href')
        except:
            continue
        r = requests.get(link, allow_redirects=True)
        headers = r.headers['content-disposition']
        filename = re.findall("filename=\"(.+)\"", headers)[0]
        open(filename, 'wb').write(r.content)
    #click next button
    driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div[4]/div[2]/div[2]/div/div[3]/a").click()

<ipython-input-33-ba979a4cc130>:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  rows = driver.find_elements_by_class_name("DJAQA0Bgt6HD8NwtPxQv")
<ipython-input-33-ba979a4cc130>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div[4]/div[2]/div[2]/div/div[3]/a").click()


PermissionError: [Errno 13] Permission denied: '2017.02.28_GM Financial Automobile Leasing Trust 2017-1 Data Tape.csv'

# Data Preprocessing

Older data were snapshots of the performance of the loans at a time period. For the purposes of predicting probability of default, I only need the most recent performance of the loans. I recognize that I could have performed this during collection, but I wanted all the data for other purposes unrelated to this analysis.

First I will filter out the files containing "Lease" or "Leasing". Leases are fundamentally different from car loan originations, so I will not consider them. 

In [43]:
for fname in os.listdir(os.getcwd()):
    if 'Leas' in fname:
        os.remove(os.path.join(os.getcwd(), fname))

What we are left with is 6856 non-lease auto ABS snapshots.

Next, I will remove all but the most recent snapshot for each of the funds. I found automating this process to be more trouble than it was worth due to inconsistent filenames, so I performed this process by manually reviewing each filename in the file system and deleting them. See below for the final list of filenames after this process:

In [48]:
for fname in os.listdir(os.getcwd()):
    print(fname)

2022.01.04_GM Financial Consumer Automobile Receivables Trust 2022-1 Data Tape.csv
2022.01.06_BMW Vehicle Owner Trust 2018-A Data Tape.csv
2022.01.06_BMW Vehicle Owner Trust 2019-A Data Tape.csv
2022.01.06_BMW Vehicle Owner Trust 2020-A Data Tape.csv
2022.01.12_CarMax Auto Owner Trust 2022-1 Data Tape.csv
2022.01.12_Ford Credit Auto Owner Trust 2022-A Data Tape.csv
2022.01.13_CarMax Auto Owner Trust 2022-1 Data Tape.csv
2022.01.14_Carvana Auto Receivables Trust 2020-P1 Data Tape.csv
2022.01.14_Carvana Auto Receivables Trust 2021-N1 Data Tape.csv
2022.01.14_Carvana Auto Receivables Trust 2021-N2 Data Tape.csv
2022.01.14_Carvana Auto Receivables Trust 2021-N3 Data Tape.csv
2022.01.14_Carvana Auto Receivables Trust 2021-N4 Data Tape.csv
2022.01.14_Carvana Auto Receivables Trust 2021-P1 Data Tape.csv
2022.01.14_Carvana Auto Receivables Trust 2021-P2 Data Tape.csv
2022.01.14_Carvana Auto Receivables Trust 2021-P3 Data Tape.csv
2022.01.14_Carvana Auto Receivables Trust 2021-P4 Data Tape.csv


The following are the total unique funds and years using only the snapshots between 1/1/2022 and 2/2/2022.
I will be removing the motorcycle fund to limit this analysis to only cars:

World Omni Auto Receivables Trust 2018-21  
World Omni Select Auto Trust 2019-21  
Toyota Auto Receivables 2018-22  
Fifth Third Auto Trust 2019  
Exeter Automobile Receivables Trust 2020-21  
Ally Auto Receivables Trust 2018-19  
<s>Harley-Davidson Motorcycle Trust 2019-21</s>  
Hyundai Auto Receivables Trust 2018-21  
Mercedez Benz Auto Receivables Trust 2018-21  
GM Financial Consumer Automobile Receivables Trust 2018-22  
AmeriCredit Automobile Receivables Trust 2017-21  
Volkswagen Auto Loan Enhanced Trust 2018,2020,2021  
Honda Auto Receivables 2018-21  
USAA Auto Owner Trust 2019  
Ford Credit Auto Owner Trust 2018-22  
Nissan Auto Receivables 2017-21  
Carvana Auto Receivables 2020-21  
CarMax Auto Owner Trust 2021-22  
Capital One Prime Auto Receivables Trust 2018-21  
Santander Drive Auto Receivables Trust 2017-21  
Drive Auto Receivables Trust 2017-21  
BMW Vehicle Owner Trust 2018-20  

This leaves us with 21 funds, 4 more than the consumer reports article identified from October 2021.  
https://www.consumerreports.org/car-financing/many-americans-overpay-for-car-loans-a8076436935/

Next we will aggregate these files into a single csv and filter to only the columns needed in our analysis